In [ ]:
from datetime import datetime, timedelta
import pandas_datareader as pdr
from finta import TA
import pandas as pd


class EquityData:

    def __init__(self):

        self.start_date = datetime.now().date() - timedelta(days=20)
        self.end_date = datetime.now().date()
        self.ticker_list = ['AAPL', 'TSLA']
        self.indicator_list = ['SMA', 'EMA']
        self.df = pd.DataFrame([])
        self.get_eod_data(self)

    @staticmethod
    def get_eod_data(self):

        """
        Get EOD data for a list of tickers
        """
        eod_df = pd.DataFrame([])

        for t in self.ticker_list:
            results = pdr.DataReader(t, 'yahoo', self.start_date, self.end_date)
            results['ticker'] = t
            eod_df = eod_df.append(results)

        # Calc the list of indicators for each set of EOD data
        eod_df = eod_df.groupby(['ticker']).apply(
                            lambda x: self.compute_analytics(x, self.indicator_list)).reset_index()

        eod_df = self.clean_data(eod_df)
        self.df = eod_df

    @staticmethod
    def compute_analytics(df, indicators):

        """
        Get calculations for each ticker in the df
        """

        for indicator in indicators:
            # Using eval so we can iterate over a list of indicators
            data = eval('TA.' + indicator + '(df)')

            # TA returns a set -> so we convert that to a data frame
            data = data.to_frame()

            # The columns names are too long so I split the string
            # and join that with the indicator name for my indicator column name
            first_string = data.columns[0].split()[0]
            data = data.rename(columns={data.columns[0]: first_string})
            data = data.add_prefix(indicator + '_')

            # Merge data frames based on the date
            df = df.merge(data, left_index=True, right_index=True)

        return df

    @staticmethod
    def clean_data(df):

        # Update the column names to follow PEP8
        df = df.rename(columns={'Date': 'date', 'High': 'high', 'Low': 'low', 'Open': 'open',
                                'Close': 'close', 'Volume': 'vol', 'Adj Close': 'adj_close',
                                'SMA_41': 'sma_41', 'EMA_9': 'ema_9'})
        return df


In [2]:
obj = EquityData()
print(obj.df)

         date        high         low        open       close          vol  \
0  2020-10-28  115.430000  111.099998  115.050003  111.199997  143937800.0   
1  2020-10-28  418.600006  406.000000  416.480011  406.019989   25451400.0   
2  2020-10-29  116.930000  112.199997  112.370003  115.320000  146129200.0   
3  2020-10-29  418.059998  406.459991  409.959991  410.829987   22655300.0   
4  2020-10-30  111.989998  107.720001  111.059998  108.860001  190272600.0   
5  2020-10-30  407.589996  379.109985  406.899994  388.040009   42511300.0   
6  2020-11-02  110.680000  107.320000  109.110001  108.769997  122866900.0   
7  2020-11-02  406.980011  392.299988  394.000000  400.510010   29021100.0   
8  2020-11-03  111.489998  108.730003  109.660004  110.440002  107624400.0   
9  2020-11-03  427.769989  406.690002  409.730011  423.899994   34351700.0   
10 2020-11-04  115.589996  112.349998  114.139999  114.949997  138235500.0   
11 2020-11-04  435.399994  417.100006  430.619995  420.980011   